In [1]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
from math import radians, cos, sin, asin, sqrt, isnan
import random
from ortools.sat.python import cp_model

In [2]:
#-----------------------------------EXTRACT THE DATA-----------------------------------

# users-melbcbd-generated.csv contains:
requests_path = '../eua-dataset/users/'
U = pd.read_csv(requests_path + 'users-test.csv') # test with 8 users

# site-optus-melbCBD.csv contains:
nodes_path = '../eua-dataset/edge-servers/'
N_src = pd.read_csv(nodes_path + 'serverstest_v2.csv') # test with 3 servers
N_dest = pd.read_csv(nodes_path + 'serverstest_v2.csv') # test with 3 servers

In [3]:
#-----------------------------------INPUTS-----------------------------------

# Incoming requests to node i
# Function(rows) x Node(column) [request/sec]
lambda_ri=[[1,0,0],[1,0,1],[1,1,1],[2,0,0]] 
#lambda_ri=[[0,0,0],[0,0,0],[0,0,0],[0,0,0]] 
# Amount of request received in time-slot
R=np.sum(lambda_ri)

# Identifies which user sent the request [U x R]
req_u =np.zeros([len(U),R])
req_u=[[1,0,0,0,0,0,0,0,0],
       [0,1,0,0,0,0,0,0,0],
       [0,0,1,0,0,0,0,0,0],
       [0,0,0,1,0,0,0,0,0],
       [0,0,0,0,1,0,0,0,0],
       [0,0,0,0,0,1,0,0,0],
       [0,0,0,0,0,0,1,0,0],
       [0,0,0,0,0,0,0,1,0]]
 
# Memory of function: m_f
m1 = 1
m2 = 2
m3 = 3
m4 = 4
m_f = [m1,m2,m3,m4]

# 1 if request r arrives to node i [N x R]
loc_arrival_r=np.zeros([len(N_dest),R])

# Show which requests are assigned to each function [F x R]
req_dist = np.zeros([len(m_f),R])

r = 0
while r<R:
    for i in range(len(N_src)):
        for f in range(len(m_f)):
            dif = lambda_ri[f][i] 
            while dif >0:
                req_dist[f][r]=1
                loc_arrival_r[i][r]=1
                r=r+1
                dif = dif-1


# Matrix that assignes a function memory to each request [F x N]
m_request = np.empty((len(m_f),R))
for f in range(len(lambda_ri)):
  for r in range (R):
        m_request[f][r] = m_f[f]*req_dist[f][r]

# Sort the requests by their memory requirement --- returns position of the [] where request is found
m_index = []

for r in range (R):
  for f in range (len(m_f)):
    if m_request[f][r]!=0:
      m_index.append(m_request[f][r])
       
final_index=np.argsort(m_index)

# Maximum allowed network delay for function f: phi_f
phi_f = [10,10,10,10]

In [4]:
#-----------------------------------INFRASTRUCTURED DATA-----------------------------------

# Memory available in node j: M_j
M_j = [10,10,10]

# Cores on node j: U_j
U_j = [10,18,18]

In [5]:
#-----------------------------------MONITORED DATA-----------------------------------
# Network delay between nodes i and j
delta_ij = []

# Cores used by node j per single request: u_fj 
# u_fj = [[2,2,2,2,2,2,2,2],[2,2,2,2,2,2,2,2],[2,2,2,2,2,2,2,2]]
u_fj = np.array([[2,2,2],[2,2,2],[2,2,2],[2,2,2]])

In [6]:
#-----------------------------------VARIABLES-----------------------------------
S_jr = [] # Set of requests conected to node n

In [7]:
#-----------------------------------HAVERSINE-----------------------------------
def haversine(lon1, lat1, lon2, lat2):
  
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [8]:
#-----------------------------------COVERAGE REQUEST-NODE-----------------------------------

#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(N_src), 0.03)

for i in range(len(N_src)):
  node_latitude = N_src.iloc[i]['LATITUDE']
  node_longitude = N_src.iloc[i]['LONGITUDE']
  temp = []
  for r in range(R):
    for u in range(len(U)):
      if req_u[u][r]==1:
        request_latitude = U.iloc[u]['Latitude']
        request_longitude = U.iloc[u]['Longitude']
        dist_geo = haversine(node_longitude, node_latitude, request_longitude, request_latitude)
        if dist_geo <= radius[i]:
          temp.append(1)
        
        else:
          temp.append(0)
       
  S_jr.append(temp)


In [9]:
#---------------------------------DISTANCE BETWEEN NODES---------------------------------

#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(N_src), 0.03)

for i in range(len(N_src)):
  node1_latitude = N_src.iloc[i]['LATITUDE']
  node1_longitude = N_src.iloc[i]['LONGITUDE']
  temp_dist = []
  for j in range(len(N_dest)):
    node2_latitude = N_dest.iloc[j]['LATITUDE']
    node2_longitude = N_dest.iloc[j]['LONGITUDE']

    dist_geo_nodes = haversine(node1_longitude, node1_latitude, node2_longitude,node2_latitude)
    temp_dist.append(np.round(dist_geo_nodes,3))
       
  delta_ij.append(temp_dist)
    

In [10]:
#-------------------------Decision variables--------------
x_jr = np.zeros(shape=(len(N_dest),R))
c_fj = np.zeros(shape=(len(m_f),len(N_dest)))
y_j = np.zeros(len(N_dest))
S_active = np.zeros(shape=(len(m_f),len(N_dest)))

In [11]:
print("INICIAL c_fj")
print(c_fj)
request_index=0
while request_index in final_index:
    r = final_index[request_index]
    index_j =np.argsort(U_j)
    index_j = index_j[::-1]
    loc=0
    print("---------------------REQUEST:  ",r," ----------------------------------")
    print("ORDER CHECK NODES:  ", index_j)
    for j in index_j:
        for f in range(len(lambda_ri)):
            if all(S_active[f,:]==0) and loc==0 and req_dist[f][r]: #Checks if there is at least one container for f active (if not goes in and turns it on)
                print(f" **** NO CONTAINERS FOR FUNCTION {f} **** ") 
                active_j=np.where(y_j==1)[0][:]  
                ordered_active_j = [x for x in index_j if x  in active_j]
                for j_temp in ordered_active_j:
                    if S_jr[j_temp][r]==1 and loc==0:
                        print("Active nodes:  ", ordered_active_j)
                        print("NODE:  ",j_temp," FUNCTION: ", f)
                        print("✓ proximity constraint ")
                        if sum(m_f[f_temp]*c_fj[f_temp,j_temp] for f_temp in range(len(lambda_ri)))+m_f[f]<=M_j[j_temp]: #memory constraint
                            print("✓ memory constraint ")
                            print("SUM MEMORY: ", sum(m_f[f_temp]*c_fj[f_temp,j_temp] for f_temp in range(len(lambda_ri)))+m_f[f])
                            if sum(x_jr[j_temp,r_temp]*u_fj[f_temp,j_temp]*req_dist[f_temp,r_temp] for f_temp in range(len(lambda_ri)) for r_temp in final_index)+u_fj[f][j_temp]*req_dist[f][r]<= U_j[j_temp]: #core constraint
                                print("✓ core constraint ")
                                print("CORE REQ: ",sum(x_jr[j_temp,r_temp]*u_fj[f_temp,j_temp]*req_dist[f_temp,r_temp] for f_temp in range(len(lambda_ri)) for r_temp in final_index)+u_fj[f][j_temp]*req_dist[f][r] )
                                for i in range(len(N_src)):
                                    if delta_ij[i][j_temp]<phi_f[f] and loc_arrival_r[i][r]==1 and req_dist[f][r]==1: #delay constraint
                                        print("✓ delay constraint, arrived to node: ", i)
                                        loc=1
                                        x_jr[j_temp][r]=1
                                        S_active[f][j_temp]=1
                                        y_j[j_temp]=1
                                        c_fj[f][j_temp]=1
                                        U_j[j_temp]=U_j[j_temp]-u_fj[f,j_temp]*req_dist[f,r]
                                        print("DEPLOY CONTAINER IN NODE: ",j_temp," function: ", f)
                                        break
                if j not in active_j and loc==0:  
                    print("NODE:  ",j," FUNCTION: ", f)
                    if S_jr[j][r]==1:
                        print("✓ proximity constraint ")
                       
                        if (sum(m_f[f_temp]*c_fj[f_temp,j] for f_temp in range(len(lambda_ri)))+m_f[f])<=(M_j[j]): #memory constraint
                            print("✓ memory constraint ")
                            print("SUM MEMORY: ", (sum(m_f[f_temp]*c_fj[f_temp,j] for f_temp in range(len(lambda_ri))))+m_f[f])
                            if (sum(x_jr[j,r_temp]*u_fj[f_temp,j]*req_dist[f_temp,r_temp] for f_temp in range(len(lambda_ri)) for r_temp in final_index)+u_fj[f][j]*req_dist[f][r])<=(U_j[j]) : #core constraint
                                print("✓ core constraint ")
                                print("CORE REQ: ",sum(x_jr[j,r_temp]*u_fj[f_temp,j]*req_dist[f_temp,r_temp] for f_temp in range(len(lambda_ri)) for r_temp in final_index)+u_fj[f][j]*req_dist[f][r])

                                for i in range(len(N_src)):
                                        if delta_ij[i][j]<phi_f[f] and loc_arrival_r[i][r]==1 and req_dist[f][r]==1: #delay constraint
                                            print("✓ delay constraint, arrived to node: ", i)
                                            loc=1
                                            x_jr[j][r]=1
                                            S_active[f][j]=1
                                            y_j[j]=1
                                            U_j[j]=U_j[j]-u_fj[f,j]*req_dist[f,r]
                                            c_fj[f][j]=1
                                            print("ACTIVATE NODE: ",j," for function: ", f)
                                            break

            if any(S_active[f,:]==1) and loc==0 and req_dist[f][r]==1: #Checks if there is at least one container for f active 
                active_loc_f=np.where(S_active[f,:]==1)[0][:]
                print(f"** ACTIVE CONTAINERS  of function {f} are located on:  ", active_loc_f)
                if S_jr[j][r]==1 and j in (active_loc_f): #Proximity constraint:
                    print(f"THERE IS AN ACTIVE CONTAINER OF TYPE {f} IN NODE {j} THAT CAN BE USED")
                    print("✓ proximity constraint ")
                    if sum(m_f[f_temp]*c_fj[f_temp,j] for f_temp in range(len(lambda_ri)))<=M_j[j]: #memory constraint
                        print("✓ memory constraint ")
                        print("SUM MEMORY: ", sum(m_f[f_temp]*c_fj[f_temp,j] for f_temp in range(len(lambda_ri))))
                        if sum(x_jr[j,r_temp]*u_fj[f_temp,j]*req_dist[f_temp,r_temp] for f_temp in range(len(lambda_ri)) for r_temp in final_index)+u_fj[f][j]*req_dist[f][r]<= U_j[j]: #core constraint
                            print("✓ core constraint ")
                            print("CORE REQ: ",sum(x_jr[j,r_temp]*u_fj[f_temp,j]*req_dist[f_temp,r_temp] for f_temp in range(len(lambda_ri)) for r_temp in final_index)+ u_fj[f][j]*req_dist[f][r] )

                            for i in range(len(N_src)):
                                if delta_ij[i][j]<phi_f[f] and loc_arrival_r[i][r]==1 and req_dist[f][r]==1: #delay constraint
                                        print("✓ delay constraint, arrived to node: ", i)
                                        loc=1
                                        x_jr[j][r]=1
                                        U_j[j]=U_j[j]-u_fj[f,j]*req_dist[f,r]
                                        print("USES NODE: ",j," function: ", f," request: ",r)
                                        break
                if S_jr[j][r]==1 and (j not in active_loc_f): #Proximity constraint:
                    print("   REQUEST: ",r,"NODE:  ",j," FUNCTION: ", f)
                    print("✓ proximity constraint ")
                    if sum(m_f[f_temp]*c_fj[f_temp,j] for f_temp in range(len(lambda_ri)))+m_f[f]<=(M_j[j]): #memory constraint
                        print("✓ memory constraint ")
                        print("SUM MEMORY: ", sum(m_f[f_temp]*c_fj[f_temp,j] for f_temp in range(len(lambda_ri)))+m_f[f])
                        if sum(x_jr[j,r_temp]*u_fj[f_temp,j]*req_dist[f_temp,r_temp] for f_temp in range(len(lambda_ri)) for r_temp in final_index)+u_fj[f][j]*req_dist[f][r]<= (U_j[j]): #core constraint
                            print("✓ core constraint ")
                            print("CORE REQ: ",sum(x_jr[j,r_temp]*u_fj[f_temp,j]*req_dist[f_temp,r_temp] for f_temp in range(len(lambda_ri)) for r_temp in final_index)+u_fj[f][j]*req_dist[f][r] )
                            for i in range(len(N_src)):
                                if delta_ij[i][j]<phi_f[f] and loc_arrival_r[i][r]==1 and req_dist[f][r]==1: #delay constraint
                                        print("✓ delay constraint, arrived to node: ", i)
                                        loc=1
                                        x_jr[j][r]=1
                                        S_active[f][j]=1
                                        y_j[j]=1
                                        U_j[j]=U_j[j]-u_fj[f,j]*req_dist[f,r]
                                        c_fj[f][j]=1
                                        #print("ENTRO EXISTING NODE j:",j," and function: ",f, "for request: ",r)
                                        print("ACTIVATE CONTAINER NODE: ",j," function: ", f," request: ",r)
                                        break

    print("Core capacity: ", U_j, "Memory capacity: ", M_j)
    print("Active nodes: " ,y_j)
    print("Coontainer allocation: ")
    print(c_fj)
    request_index=request_index+1

print("---------------------------------------------------")
for f in range (len(m_f)):
    
    
    if all(c_fj[f,:]==0) and any(y_j==1): #use nodes that are already being used
        rand_node= np.where(y_j==1)[0][:]  
        for t in rand_node:
            if sum(m_f[f_temp]*c_fj[f_temp,t] for f_temp in range(len(lambda_ri)))<=(M_j[t]*y_j[t]):
                c_fj[f][rand_node]=1
                S_active[f][rand_node]=1
                y_j[rand_node]=1
    
    if all(c_fj[f,:]==0): #when there are no requests
        rand_node= np.random.choice(range(len(N_src)))
        c_fj[f][rand_node]=1
        S_active[f][rand_node]=1
        y_j[rand_node]=1
    

print(c_fj)

INICIAL c_fj
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
---------------------REQUEST:   0  ----------------------------------
ORDER CHECK NODES:   [2 1 0]
 **** NO CONTAINERS FOR FUNCTION 0 **** 
NODE:   2  FUNCTION:  0
✓ proximity constraint 
✓ memory constraint 
SUM MEMORY:  1.0
✓ core constraint 
CORE REQ:  2.0
✓ delay constraint, arrived to node:  0
ACTIVATE NODE:  2  for function:  0
Core capacity:  [10, 18, 16.0] Memory capacity:  [10, 10, 10]
Active nodes:  [0. 0. 1.]
Coontainer allocation: 
[[0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
---------------------REQUEST:   1  ----------------------------------
ORDER CHECK NODES:   [1 2 0]
 **** NO CONTAINERS FOR FUNCTION 1 **** 
NODE:   1  FUNCTION:  1
✓ proximity constraint 
✓ memory constraint 
SUM MEMORY:  2.0
✓ core constraint 
CORE REQ:  2.0
✓ delay constraint, arrived to node:  0
ACTIVATE NODE:  1  for function:  1
Core capacity:  [10, 16.0, 16.0] Memory capacity:  [10, 10, 10]
Active nodes:  [0. 1. 1.]
Coontainer allo

In [12]:
print("CORE REQ: ",sum(x_jr[0,r_temp]*u_fj[f_temp,0]*req_dist[f_temp,r_temp] for f_temp in range(len(lambda_ri)) for r_temp in final_index)+u_fj[f][0]*req_dist[f][r] )

CORE REQ:  2.0


In [13]:
#-----------------------------------Normalization-----------------------------------
# mat_mul = np.dot(req_dist,np.transpose(x_rj))
# x_fj = np.zeros([len(lambda_ri),len(N_dest)])
# for j in range(len(N_dest)):
#     for f in range(len(lambda_ri)):
#         if sum(mat_mul[f])==0:
#             x_fj[f][j] = 0
#         else:
#             x_fj[f][j] = mat_mul[f][j]/sum(mat_mul[f])

# print(x_fj)